In [69]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
from scipy.stats import norm
#import statsmodels.stats as sm

import statsmodels.api as sm

import hashlib
%matplotlib inline

sns.set(style="dark")
plt.style.use("ggplot")

In [70]:
pre_life = pd.read_csv('~/Downloads/Life Expectancy Data.csv') #2000 - 2015
life = pre_life.dropna()
pre_happy = pd.read_csv("~/Downloads/World Happiness Report.csv") #between 2005 and 2022
happy = pre_happy.dropna()

## T-Test

In [72]:
merged = life.merge(happy, how='inner', left_on='Country', right_on='Country Name')

In [73]:
# columns that we will be looking at for association with life_expectancy
X = ['Log GDP Per Capita', 'Social Support', 'Freedom To Make Life Choices', 'Generosity', 'Perceptions Of Corruption', 'Confidence In National Government']

In [74]:
def fit_OLS_model(df, target_variable, explanatory_variables, intercept = False):
    """
    Fits an OLS model from data.
    
    Inputs:
        df: pandas DataFrame
        target_variable: string, name of the target variable
        explanatory_variables: list of strings, names of the explanatory variables
        intercept: bool, if True add intercept term
    Outputs:
        fitted_model: model containing OLS regression results
    """
    
    target = df[target_variable]
    inputs = df[explanatory_variables]
    if intercept:
        inputs = sm.add_constant(inputs)
    
    fitted_model = sm.OLS(target, inputs).fit()
    return(fitted_model)

def mean_squared_error(true_vals, predicted_vals):
    """
    Return the mean squared error
    
    Inputs:
        true_vals: array of true labels
        predicted_vals: array labels predicted from the data
    Output:
        float, mean squared error of the predicted values
    """
    return np.mean((true_vals - predicted_vals) ** 2)

In [85]:
linear_model = fit_OLS_model(merged, 'Life expectancy ', ['Social Support', 'Freedom To Make Life Choices', 'Generosity', 'Perceptions Of Corruption', 'Confidence In National Government', 'Log GDP Per Capita'])
tvalues = linear_model.tvalues

Social Support                        -9.889543
Freedom To Make Life Choices           8.372626
Generosity                            13.982626
Perceptions Of Corruption              5.708978
Confidence In National Government    -12.977188
Log GDP Per Capita                   169.614351
dtype: float64

## Benjamini Yekutieli Correction

In [49]:
def benjamini_yekutieli(p_values, alpha):
    """
    Returns decisions on p-values using Benjamini-Hochberg.
    
    Inputs:
        p_values: array of p-values
        alpha: desired FDR (FDR = E[# false positives / # positives])
    
    Returns:
        decisions: binary array of same length as p-values, where `decisions[i]` is 1
        if `p_values[i]` is deemed significant, and 0 otherwise
    """
    n = len(p_values)
    c = np.sum([1/i for i in np.arange(n)]) #is n the number of samples, or the num of p-values
    assorted_p = sorted(p_values)
    line = sorted([k*(alpha)/(n * c) for k in range(1, len(assorted_p) + 1)])
    
    chosen_p = 0 
    for i in range(len(assorted_p)):
        if line[i] > assorted_p[i]:
            chosen_p = i
    
    decisions = np.array(assorted_p[chosen_p] >= p_values)
    return decisions.astype(int)

In [37]:
# for now: alpha = 0.05 just because
decision_dict = {}
for key, values in p_dict.items():
    decisions = benjamini_hochberg(values, 0.05)
    decision_dict[key] = decisions

# Bonferroni Correction

In [41]:
def bonferroni(p_values, alpha_total):
    """
    Returns decisions on p-values using the Bonferroni correction.
    
    Inputs:
        p_values: array of p-values
        alpha_total: desired family-wise error rate (FWER = P(at least one false discovery))
    
    Returns:
        decisions: binary array of same length as p-values, where `decisions[i]` is 1
        if `p_values[i]` is deemed significant, and 0 otherwise
    """
    decisions = [(alpha_total)/len(p_values) >= p for p in p_values]
    return decisions

In [42]:
decision_dict = {}
for key, values in p_dict.items():
    decisions = bonferroni(values, 0.05)
    decision_dict[key] = decisions